In [1]:
import spacy
import pandas as pd
import numpy as np
embedder = spacy.load("en_core_web_md")

In [10]:
dataset = pd.read_csv('Language Detection.csv')
x = dataset['Text'].to_numpy()
y = dataset['Language'].to_numpy()


In [12]:
x = [embedder(converted_string) for converted_string in x]
x = [v.vector for v in x]


In [13]:
#train test split the data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [14]:
# make a random forest classifier
from sklearn.ensemble import RandomForestClassifier

def make_classifier(reviews , labels):
    clf = RandomForestClassifier(n_estimators=100 , random_state=0 )
    clf.fit(reviews , labels)
    return clf
classifier_ = make_classifier(x_train , y_train)

In [15]:
print(classifier_.score(x_train , y_train))
print(classifier_.score(x_test , y_test))

0.9383552176917761
0.8497743391360413


In [16]:
from sklearn.ensemble import GradientBoostingClassifier
classifier_2 = GradientBoostingClassifier(n_estimators=100 , random_state=0)
classifier_2.fit(x_train , y_train)


0.9383552176917761


In [19]:
print(classifier_2.score(x_test , y_test))

0.8455834945196647


In [28]:
x = [embedder(converted_string) for converted_string in ['ich bin aus ayland']]
x = [v.vector for v in x]


In [36]:
classifier_2.predict_proba(x).tolist()[0]

[0.00019575892059226898,
 6.536186697270984e-05,
 3.930936781640891e-05,
 3.061978382699012e-06,
 4.158852407443805e-06,
 0.9994174404434346,
 6.26690259735485e-06,
 2.013547670235933e-09,
 9.641588784006352e-06,
 1.1325144844053207e-05,
 4.210104684632446e-05,
 3.224614173073272e-05,
 3.17418232069675e-06,
 3.4535746343522256e-05,
 7.783458593003339e-05,
 3.1748681779357106e-05,
 2.6032535670193618e-05]

In [27]:
classifier_2.classes_

array(['Arabic', 'Danish', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Hindi', 'Italian', 'Kannada', 'Malayalam', 'Portugeese',
       'Russian', 'Spanish', 'Sweedish', 'Tamil', 'Turkish'], dtype=object)

In [37]:
dict(zip(classifier_2.classes_ , classifier_2.predict_proba(x).tolist()[0]))

{'Arabic': 0.00019575892059226898,
 'Danish': 6.536186697270984e-05,
 'Dutch': 3.930936781640891e-05,
 'English': 3.061978382699012e-06,
 'French': 4.158852407443805e-06,
 'German': 0.9994174404434346,
 'Greek': 6.26690259735485e-06,
 'Hindi': 2.013547670235933e-09,
 'Italian': 9.641588784006352e-06,
 'Kannada': 1.1325144844053207e-05,
 'Malayalam': 4.210104684632446e-05,
 'Portugeese': 3.224614173073272e-05,
 'Russian': 3.17418232069675e-06,
 'Spanish': 3.4535746343522256e-05,
 'Sweedish': 7.783458593003339e-05,
 'Tamil': 3.1748681779357106e-05,
 'Turkish': 2.6032535670193618e-05}

In [41]:
#save the model to pickle file
import joblib
from joblib import dump, load
joblib.dump(classifier_ , 'classifier.pkl')
joblib.dump(classifier_2 , 'classifier_2.pkl')

['classifier_2.pkl']

In [42]:
model = joblib.load('classifier_2.pkl')

In [44]:
def pred(text):
    x = [embedder(converted_string) for converted_string in [text]]
    x = [v.vector for v in x]
    return dict(zip(classifier_2.classes_ , classifier_2.predict_proba(x).tolist()[0]))

In [48]:
import gradio as gr
text_input = gr.inputs.Textbox(lines=3)
label = gr.outputs.Label()
examples = ['اهلا و سهلا , مرحبا' , 
            'hello , how are you my name is omar , iam from egypt and i really like gaming' ,
            'Доброе утро , Здравствуйте , Меня зовут Мандли' , 
            'Bonjour , Comment allez-vous , S’il vous plaît , Je m’appelle , Je ne comprends pas']
intf = gr.Interface(fn = pred , inputs = text_input , outputs = label , examples=examples)
intf.launch(share = True)

c:\Users\antoz\anaconda3\lib\site-packages\gradio\inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\antoz\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\Users\antoz\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\Users\antoz\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
c:\Users\antoz\anaconda3\lib\site-packages\gradio\outputs.py:196: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://25845.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x1d228204a00>,
 'http://127.0.0.1:7860/',
 'https://25845.gradio.app')

In [73]:
np.array(x_train)[0].shape

(300,)

In [81]:
import tensorflow as tf

In [92]:
# create a neural network with 2 layers

from keras.models import Sequential
from keras.layers import Dense ,BatchNormalization
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=(300,) )   ) 
model.add(BatchNormalization())
model.add(Dense(units=100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=17, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(np.array(x_train), y_train_one_hot, epochs=10, batch_size=100 ,validation_data=(np.array(x_test) , y_test_one_hot))

Epoch 1/10
73/73 [==============================] - 2s 10ms/step - loss: 0.9861 - accuracy: 0.7028 - val_loss: 1.5160 - val_accuracy: 0.6812
Epoch 2/10
73/73 [==============================] - 0s 6ms/step - loss: 0.4658 - accuracy: 0.8415 - val_loss: 0.7268 - val_accuracy: 0.8095
Epoch 3/10
73/73 [==============================] - 0s 5ms/step - loss: 0.3886 - accuracy: 0.8668 - val_loss: 0.4915 - val_accuracy: 0.8404
Epoch 4/10
73/73 [==============================] - 0s 6ms/step - loss: 0.3550 - accuracy: 0.8745 - val_loss: 0.4282 - val_accuracy: 0.8617
Epoch 5/10
73/73 [==============================] - 0s 5ms/step - loss: 0.3291 - accuracy: 0.8809 - val_loss: 0.4412 - val_accuracy: 0.8504
Epoch 6/10
73/73 [==============================] - 0s 6ms/step - loss: 0.3153 - accuracy: 0.8846 - val_loss: 0.4268 - val_accuracy: 0.8623
Epoch 7/10
73/73 [==============================] - 0s 6ms/step - loss: 0.3015 - accuracy: 0.8898 - val_loss: 0.4459 - val_accuracy: 0.8540
Epoch 8/10
73/73 [=

In [67]:
#one hot encode y_train
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_train_one_hot = lb.fit_transform(y_train)

In [76]:
y_test_one_hot = lb.transform(y_test)